# 0) Import

In [4]:
import numpy as np 
import networkx as nx
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.preprocessing import Normalizer, MinMaxScaler
from scipy.sparse import csgraph 
import scipy
import os
import random
from sklearn import datasets
import matplotlib.pyplot as plt
from matplotlib import style
#style.use("ggplot")
import time
from datetime import datetime

# 1) Graph & Laplacian

In [5]:
def print_graph(ADJOINT_MTX):
    print(np.round(ADJOINT_MTX, 3))

In [6]:
def create_graph_from_adjoint_matrix(NODE_NUM, ADJOINT_MTX):
    output = nx.Graph()
    temp = list(range(NODE_NUM))    
    output.add_nodes_from(temp) 
    for i in temp:
        for j in temp:
            if ADJOINT_MTX[i,j] == 0:
                pass
            else:
                output.add_edges(i,j, weight = ADJOINT_MTX[i,j])   
    return output

In [7]:
def create_ER_graph(NODE_NUM, PROB):
    temp = nx.erdos_renyi_graph(NODE_NUM, PROB)
    output = nx.to_numpy_array(temp)
    return output

In [8]:
def create_RBF_graph(NODE_NUM, DIMENSION, THRESHOLD):
    temp = np.random.normal(size = (NODE_NUM, DIMENSION))
    output = rbf_kernel(temp, gamma = 0.05)
    output[output < THRESHOLD] = 0
    return output

In [9]:
lap_types = ['combinatorial', 'symmetric_normalized', 'random_walk']

def graph_to_laplacian(ADJOINT_MTX, LAPLACIAN_TYPE):  
    NUM = ADJOINT_MTX.shape[0] # square라 ok
    if LAPLACIAN_TYPE not in lap_types:
        print('wrong type !!')
        return

    D_list = np.sum(ADJOINT_MTX, axis = 1)
    D_inv_list = [1/i if i > 0 else 0 for i in D_list ]
    sqrt_D_list = [i**(0.5) for i in D_list]
    sqrt_D_inv_list = [1/i if i > 0 else 0 for i in sqrt_D_list]
    
    D = np.diag(D_list)
    L = D - ADJOINT_MTX

    if LAPLACIAN_TYPE == 'combinatorial': 
        output = L
    elif LAPLACIAN_TYPE == 'symmetric_normalized':
        output = np.empty((NUM, NUM))
        for i in range(0, NUM):
            for j in range(0, NUM):
                output[i,j] = sqrt_D_inv_list[i] * L[i,j] * sqrt_D_inv_list[j]
    else: # random_walk
        output = np.empty((NUM, NUM))
        for i in range(0, NUM):
            for j in range(0, NUM):
                output[i,j] = D_inv_list[i] * L[i,j]

    np.fill_diagonal(output, 1)   
    return output

# 2) Arm features and user features

In [10]:
def create_arms_features_normal(ARM_NUM, DIMENSION):
    output = np.random.normal(size = (ARM_NUM, DIMENSION))
    output = Normalizer().fit_transform(output)
    return output

In [11]:
def create_arms_features_sparse(N,d):  
    d_p=int(d/N)
    Bs=list()
    for i in range(N):
        Bs.append(np.zeros(d_p*N))
        x=np.random.normal(0,1,d_p)
        u=np.random.uniform(0,1,1)
        b=u**(1./d_p)*x/np.sqrt(np.sum(x**2))
        Bs[i][(i*d_p):((i+1)*d_p)]=b          
    
    return Bs

In [12]:
def create_users_features(USER_NUM, DIMENSION, LAPLACIAN, GAMMA):  
    Theta0 = np.random.normal(size = (USER_NUM, DIMENSION))
    eye_plus_gamma_L = np.eye(USER_NUM) + GAMMA * LAPLACIAN
    Theta = np.matmul( np.linalg.pinv(eye_plus_gamma_L) , Theta0 )
    Theta = Normalizer().fit_transform(Theta)
    return Theta

# 3) Algorithms

## ** RidgeTS

In [12]:
class RidgeGraphThompson():
    def __init__(self, USER_NUM, ARM_NUM, DIMENSION, DELTA, OUR_GRAPH, OUR_LAPL,CONST_R, CONST_LAMBDA, CONST_V, TIME_HORIZON):
        self.user_num = USER_NUM
        self.arm_num = ARM_NUM
        self.dim = DIMENSION
        self.delta = DELTA
        self.our_graph = OUR_GRAPH
        self.our_lapl = OUR_LAPL  
        self.R = CONST_R
        self.T = TIME_HORIZON
        self.Lambda = CONST_LAMBDA
        self.v = CONST_V
        #if self.v == 0:
        #    self.v = 2 * self.R * np.sqrt( 9 * self.dim * (np.log(self.T / self.Lambda) + 1 / self.Lambda) )
        self.users_B_list = [self.Lambda * self.our_lapl[i, i] * np.identity(self.dim) for i in range(0, self.user_num)] # d x d for each user
        self.users_B_inv_list = [(1/self.Lambda) * (1/self.our_lapl[i, i]) * np.identity(self.dim) for i in range(0, self.user_num)]
        self.users_y_list = [np.zeros(self.dim) for i in range(0, self.user_num)] # 1 x d for each user
        self.users_mu_bar_list = [np.zeros(self.dim) for i in range(0, self.user_num)]
    
    def choose_arm(self, SELECTED_USER, arms_features): 
        # mu_hat
        B_INV=self.users_B_inv_list[SELECTED_USER]
        temp = 0 
        for i in range(0, self.user_num):
            temp += self.our_lapl[SELECTED_USER, i] * self.users_mu_bar_list[i]
        temp -= self.our_lapl[SELECTED_USER, SELECTED_USER] * self.users_mu_bar_list[SELECTED_USER]

        mu_hat = self.users_mu_bar_list[SELECTED_USER] - self.Lambda * np.matmul(B_INV, temp)        
        
        #mu_tide
        # construct covariance matrix
        B_CURR = self.users_B_list[SELECTED_USER] + self.Lambda * self.our_lapl[i, i] * np.identity(self.dim) 
        for i in range(0, self.user_num):
            if (i != SELECTED_USER):
                B_CURR += ((self.Lambda * self.our_lapl[SELECTED_USER, i])**2) * self.users_B_inv_list[i]
        B_CURR_INV = np.linalg.pinv(B_CURR)
        
        mu_tilde= np.random.multivariate_normal(mean = mu_hat, cov = (self.v ** 2) * B_CURR_INV)     
        est_reward=[np.dot(arms_features[i],mu_tilde) for i in range(self.arm_num)]
        chosen_arm = est_reward.index(max(est_reward))
        self.chosen_arm_feature = arms_features[chosen_arm]
        self.SELECTED_USER=SELECTED_USER
        return chosen_arm


    def update_B_and_y_and_mu_bar(self, OBSERVED_REWARD):
        self.users_B_list[self.SELECTED_USER] = self.users_B_list[self.SELECTED_USER] + np.outer(self.chosen_arm_feature, self.chosen_arm_feature)
        temp_Binv=np.copy(self.users_B_inv_list[self.SELECTED_USER])
        temp_armf=np.copy(self.chosen_arm_feature)
        #self.users_B_inv_list[self.SELECTED_USER] = np.linalg.pinv(self.users_B_list[self.SELECTED_USER])
        self.users_B_inv_list[self.SELECTED_USER]=temp_Binv-((temp_Binv).dot(np.outer(temp_armf,temp_armf))).dot(temp_Binv)/(1.+np.dot(temp_armf,(temp_Binv).dot(temp_armf)))
        self.users_y_list[self.SELECTED_USER] =  self.users_y_list[self.SELECTED_USER] + OBSERVED_REWARD * self.chosen_arm_feature
        self.users_mu_bar_list[self.SELECTED_USER] = np.matmul(self.users_B_inv_list[self.SELECTED_USER], self.users_y_list[self.SELECTED_USER])


## ** SemiRidgeTS

In [13]:
class SemiRidgeGraphThompson():
    def __init__(self, USER_NUM, ARM_NUM, DIMENSION, DELTA, OUR_GRAPH, OUR_LAPL,CONST_R, CONST_LAMBDA, CONST_V, TIME_HORIZON):       
        self.user_num = USER_NUM
        self.arm_num = ARM_NUM
        self.dim = DIMENSION
        self.delta = DELTA
        self.our_graph = OUR_GRAPH
        self.our_lapl = OUR_LAPL
        self.R = CONST_R
        self.T = TIME_HORIZON
        self.Lambda = CONST_LAMBDA
        self.v = CONST_V
        #if self.v == 0:
        #    self.v = 2 * self.R * np.sqrt( 9 * self.dim * (np.log(self.T / self.Lambda) + 1 / self.Lambda) )
        self.users_B_list = [self.Lambda * self.our_lapl[i, i] * np.identity(self.dim) for i in range(0, self.user_num)]
        self.users_B_inv_list = [(1/self.Lambda) * (1/self.our_lapl[i, i]) * np.identity(self.dim) for i in range(0, self.user_num)]
        self.users_y_list = [np.zeros(self.dim) for i in range(0, self.user_num)]
        self.users_mu_bar_list = [np.zeros(self.dim) for i in range(0, self.user_num)]
    
    def choose_arm(self, SELECTED_USER, arms_features):
        # mu_hat
        B_INV=self.users_B_inv_list[SELECTED_USER]
        temp = 0 
        for i in range(0, self.user_num):
            temp += self.our_lapl[SELECTED_USER, i] * self.users_mu_bar_list[i]
        temp -= self.our_lapl[SELECTED_USER, SELECTED_USER] * self.users_mu_bar_list[SELECTED_USER]
        
        self.mu_hat = self.users_mu_bar_list[SELECTED_USER] - self.Lambda * np.matmul(B_INV, temp)  
        
        # mu_tilde
        # construct covariance matrix
        B_CURR = self.users_B_list[SELECTED_USER] + self.Lambda * self.our_lapl[i, i] * np.identity(self.dim) 
        for i in range(0, self.user_num):
            if (i != SELECTED_USER):
                B_CURR += ((self.Lambda * self.our_lapl[SELECTED_USER, i])**2) * self.users_B_inv_list[i]
        B_CURR_INV = np.linalg.pinv(B_CURR)
        
        self.V=(self.v ** 2) * B_CURR_INV
        mu_tilde= np.random.multivariate_normal(mean = self.mu_hat, cov = self.V)     
        est_reward=[np.dot(arms_features[i],mu_tilde) for i in range(self.arm_num)]
        chosen_arm = est_reward.index(max(est_reward))
        self.chosen_arm_feature = arms_features[chosen_arm]
        self.SELECTED_USER=SELECTED_USER
        self.arms_features=arms_features
        return chosen_arm

    def update_B_and_y_and_mu_bar(self, OBSERVED_REWARD):
        mu_mc=np.random.multivariate_normal(self.mu_hat,self.V,1000)
        est_mc=list((np.dot(self.arms_features,mu_mc.T)).T) 
        ac_mc=list(np.argmax(est_mc,axis=1))
        pi_est=np.array([float(ac_mc.count(i))/len(ac_mc) for i in range(self.arm_num)])
        b_mean=np.dot(np.transpose(np.array(self.arms_features)),pi_est)
        
        self.users_B_list[self.SELECTED_USER] = self.users_B_list[self.SELECTED_USER] + np.outer(self.chosen_arm_feature-b_mean,
                                                                                                 self.chosen_arm_feature-b_mean)
        self.users_B_list[self.SELECTED_USER] = self.users_B_list[self.SELECTED_USER] + np.dot(np.dot(np.transpose(self.arms_features),np.diag(pi_est)),self.arms_features)-np.outer(b_mean,b_mean)
        
        temp_Binv=np.copy(self.users_B_inv_list[self.SELECTED_USER])
        temp_armf=np.copy(self.chosen_arm_feature-b_mean)
        self.users_B_inv_list[self.SELECTED_USER]=temp_Binv-((temp_Binv).dot(np.outer(temp_armf,temp_armf))).dot(temp_Binv)/(1.+np.dot(temp_armf,(temp_Binv).dot(temp_armf)))
        for i in range(self.arm_num):
            temp_Binv=np.copy(self.users_B_inv_list[self.SELECTED_USER])
            temp_armf=np.sqrt(pi_est[i])*(self.arms_features[i]-b_mean)
            self.users_B_inv_list[self.SELECTED_USER]=temp_Binv-((temp_Binv).dot(np.outer(temp_armf,temp_armf))).dot(temp_Binv)/(1.+np.dot(temp_armf,(temp_Binv).dot(temp_armf)))
        
        self.users_y_list[self.SELECTED_USER] =  self.users_y_list[self.SELECTED_USER] + 2*OBSERVED_REWARD * (self.chosen_arm_feature-b_mean)
        self.users_mu_bar_list[self.SELECTED_USER] = np.matmul(self.users_B_inv_list[self.SELECTED_USER], self.users_y_list[self.SELECTED_USER])
        

## SemiTS_Single

In [ ]:
class SemiTS_Single():
    def __init__(self, USER_NUM, ARM_NUM, DIMENSION, DELTA, OUR_GRAPH, OUR_LAPL,CONST_R, CONST_LAMBDA, CONST_V, TIME_HORIZON):       
        self.user_num = USER_NUM
        self.arm_num = ARM_NUM
        self.dim = DIMENSION
        self.delta = DELTA
        self.our_graph = OUR_GRAPH
        self.our_lapl = OUR_LAPL
        self.R = CONST_R
        self.T = TIME_HORIZON
        self.Lambda = CONST_LAMBDA
        self.v = CONST_V

        self.B=np.eye(self.dim)
        self.B_inv=np.eye(self.dim)
        self.y=np.zeros(self.dim)
        self.mu_hat=np.zeros(self.dim)
        
    def choose_arm(self, SELECTED_USER, arms_features):
        B_INV=self.B_inv
        self.V=(self.v ** 2) * B_INV
        mu_tilde= np.random.multivariate_normal(mean = self.mu_hat, cov = self.V)     
        est_reward=[np.dot(arms_features[i],mu_tilde) for i in range(self.arm_num)]
        chosen_arm = est_reward.index(max(est_reward))
        self.chosen_arm_feature = arms_features[chosen_arm]
        #self.SELECTED_USER=SELECTED_USER
        self.arms_features=arms_features
        return chosen_arm

    def update_B_and_y_and_mu_bar(self, OBSERVED_REWARD):
        mu_mc=np.random.multivariate_normal(self.mu_hat,self.V,1000)
        est_mc=list((np.dot(self.arms_features,mu_mc.T)).T) 
        ac_mc=list(np.argmax(est_mc,axis=1))
        pi_est=np.array([float(ac_mc.count(i))/len(ac_mc) for i in range(self.arm_num)])
        b_mean=np.dot(np.transpose(np.array(self.arms_features)),pi_est)
        
        self.B = self.B + np.outer(self.chosen_arm_feature-b_mean, self.chosen_arm_feature-b_mean)
        self.B = self.B + np.dot(np.dot(np.transpose(self.arms_features),np.diag(pi_est)),self.arms_features)-np.outer(b_mean,b_mean)
        
        temp_Binv=np.copy(self.B_inv)
        temp_armf=np.copy(self.chosen_arm_feature-b_mean)
        self.B_inv=temp_Binv-((temp_Binv).dot(np.outer(temp_armf,temp_armf))).dot(temp_Binv)/(1.+np.dot(temp_armf,(temp_Binv).dot(temp_armf)))
        for i in range(self.arm_num):
            temp_Binv=np.copy(self.B_inv)
            temp_armf=np.sqrt(pi_est[i])*(self.arms_features[i]-b_mean)
            self.B_inv=temp_Binv-((temp_Binv).dot(np.outer(temp_armf,temp_armf))).dot(temp_Binv)/(1.+np.dot(temp_armf,(temp_Binv).dot(temp_armf)))
        
        self.y =  self.y + 2*OBSERVED_REWARD * (self.chosen_arm_feature-b_mean)
        self.mu_hat = np.matmul(self.B_inv, self.y)      

## Individual Semi Ridge Graph THompson (Kim and Paik 2019)

In [ ]:
class IndividualSemiRidgeGraphThompson():
    def __init__(self, USER_NUM, ARM_NUM, DIMENSION, DELTA, OUR_GRAPH, OUR_LAPL,CONST_R, CONST_LAMBDA, CONST_V, TIME_HORIZON):       
        self.user_num = USER_NUM
        self.arm_num = ARM_NUM
        self.dim = DIMENSION
        self.delta = DELTA
        self.our_graph = OUR_GRAPH
        self.our_lapl = OUR_LAPL
        self.R = CONST_R
        self.T = TIME_HORIZON
        self.Lambda = CONST_LAMBDA
        self.v = CONST_V

        self.users_B_list = [np.identity(self.dim) for i in range(0, self.user_num)]
        self.users_B_inv_list = [np.identity(self.dim) for i in range(0, self.user_num)]
        self.users_y_list = [np.zeros(self.dim) for i in range(0, self.user_num)]
    
    def choose_arm(self, SELECTED_USER, arms_features):
        B_INV=self.users_B_inv_list[SELECTED_USER]
        
        self.mu_hat = np.matmul(B_INV, self.users_y_list[SELECTED_USER])
        
        self.V=(self.v ** 2) * B_INV
        mu_tilde= np.random.multivariate_normal(mean = self.mu_hat, cov = self.V)     
        est_reward=[np.dot(arms_features[i],mu_tilde) for i in range(self.arm_num)]
        chosen_arm = est_reward.index(max(est_reward))
        self.chosen_arm_feature = arms_features[chosen_arm]
        self.SELECTED_USER=SELECTED_USER
        self.arms_features=arms_features
        return chosen_arm

    def update_B_and_y_and_mu_bar(self, OBSERVED_REWARD):
        mu_mc=np.random.multivariate_normal(self.mu_hat,self.V,1000)
        est_mc=list((np.dot(self.arms_features,mu_mc.T)).T) 
        ac_mc=list(np.argmax(est_mc,axis=1))
        pi_est=np.array([float(ac_mc.count(i))/len(ac_mc) for i in range(self.arm_num)])
        b_mean=np.dot(np.transpose(np.array(self.arms_features)),pi_est)
        
        self.users_B_list[self.SELECTED_USER] = self.users_B_list[self.SELECTED_USER] + np.outer(self.chosen_arm_feature-b_mean,
                                                                                                 self.chosen_arm_feature-b_mean)
        self.users_B_list[self.SELECTED_USER] = self.users_B_list[self.SELECTED_USER] + np.dot(np.dot(np.transpose(self.arms_features),np.diag(pi_est)),self.arms_features)-np.outer(b_mean,b_mean)
        
        temp_Binv=np.copy(self.users_B_inv_list[self.SELECTED_USER])
        temp_armf=np.copy(self.chosen_arm_feature-b_mean)
        self.users_B_inv_list[self.SELECTED_USER]=temp_Binv-((temp_Binv).dot(np.outer(temp_armf,temp_armf))).dot(temp_Binv)/(1.+np.dot(temp_armf,(temp_Binv).dot(temp_armf)))
        for i in range(self.arm_num):
            temp_Binv=np.copy(self.users_B_inv_list[self.SELECTED_USER])
            temp_armf=np.sqrt(pi_est[i])*(self.arms_features[i]-b_mean)
            self.users_B_inv_list[self.SELECTED_USER]=temp_Binv-((temp_Binv).dot(np.outer(temp_armf,temp_armf))).dot(temp_Binv)/(1.+np.dot(temp_armf,(temp_Binv).dot(temp_armf)))
        
        self.users_y_list[self.SELECTED_USER] =  self.users_y_list[self.SELECTED_USER] + 2*OBSERVED_REWARD * (self.chosen_arm_feature-b_mean)
        

## IndivTS

In [14]:
class IndividualThompson():
    def __init__(self, USER_NUM, ARM_NUM, DIMENSION, DELTA, CONST_R, CONST_V, CONST_EPSILON, TIME_HORIZON):
        self.user_num = USER_NUM
        self.arm_num = ARM_NUM
        self.dim = DIMENSION
        self.delta = DELTA
        self.R = CONST_R
        self.T = TIME_HORIZON
        self.epsilon = CONST_EPSILON
        self.v = CONST_V
        self.users_B_list = [np.identity(self.dim) for i in range(0, self.user_num)] 
        self.users_B_inv_list = [np.identity(self.dim) for i in range(0, self.user_num)]
        self.users_y_list = [np.zeros(self.dim) for i in range(0, self.user_num)] 

        if self.v == 0:
            self.v = self.R * np.sqrt( 24 / self.epsilon * np.log(1/self.delta) )
    
    def choose_arm(self, SELECTED_USER, arms_features):
        B_INV=self.users_B_inv_list[SELECTED_USER]
        mu_hat = np.matmul(B_INV, self.users_y_list[SELECTED_USER])        
        mu_tilde= np.random.multivariate_normal(mean = mu_hat, cov = (self.v ** 2) * B_INV)     
        est_reward=[np.dot(arms_features[i],mu_tilde) for i in range(self.arm_num)]
        chosen_arm = est_reward.index(max(est_reward))
        self.chosen_arm_feature = arms_features[chosen_arm]  
        self.SELECTED_USER=SELECTED_USER 
        return chosen_arm
    
    def update_B_and_y_and_mu_bar(self, OBSERVED_REWARD):
        self.users_B_list[self.SELECTED_USER] = self.users_B_list[self.SELECTED_USER] + np.outer(self.chosen_arm_feature, self.chosen_arm_feature)
        temp_Binv=np.copy(self.users_B_inv_list[self.SELECTED_USER])
        temp_armf=np.copy(self.chosen_arm_feature)
        #self.users_B_inv_list[self.SELECTED_USER] = np.linalg.pinv(self.users_B_list[self.SELECTED_USER])
        self.users_B_inv_list[self.SELECTED_USER]=temp_Binv-((temp_Binv).dot(np.outer(temp_armf,temp_armf))).dot(temp_Binv)/(1.+np.dot(temp_armf,(temp_Binv).dot(temp_armf)))       
        self.users_y_list[self.SELECTED_USER] =  self.users_y_list[self.SELECTED_USER] + OBSERVED_REWARD * self.chosen_arm_feature


## LinTS_Single

In [ ]:
class LinTS_Single():
    def __init__(self, USER_NUM, ARM_NUM, DIMENSION, DELTA, CONST_R, CONST_V, CONST_EPSILON, TIME_HORIZON):
        self.user_num = USER_NUM
        self.arm_num = ARM_NUM
        self.dim = DIMENSION
        self.delta = DELTA
        self.R = CONST_R
        self.T = TIME_HORIZON
        self.epsilon = CONST_EPSILON
        self.v = CONST_V
        #self.users_B_list = [np.identity(self.dim) for i in range(0, self.user_num)] 
        #self.users_B_inv_list = [np.identity(self.dim) for i in range(0, self.user_num)]
        #self.users_y_list = [np.zeros(self.dim) for i in range(0, self.user_num)] 
        self.user_B = np.identity(self.dim)
        self.user_B_inv = np.identity(self.dim)
        self.user_y = np.zeros(self.dim)

        if self.v == 0:
            self.v = self.R * np.sqrt( 24 / self.epsilon * np.log(1/self.delta) )
    
    def choose_arm(self, arms_features):
        B_INV=self.user_B_inv
        mu_hat = np.matmul(B_INV, self.user_y)        
        mu_tilde= np.random.multivariate_normal(mean = mu_hat, cov = (self.v ** 2) * B_INV)     
        est_reward=[np.dot(arms_features[i],mu_tilde) for i in range(self.arm_num)]
        chosen_arm = est_reward.index(max(est_reward))
        self.chosen_arm_feature = arms_features[chosen_arm] 
        return chosen_arm
    
    def update_B_and_y_and_mu_bar(self, OBSERVED_REWARD):
        self.user_B = self.user_B + np.outer(self.chosen_arm_feature, self.chosen_arm_feature)
        temp_Binv = np.copy(self.user_B_inv)
        temp_armf = np.copy(self.chosen_arm_feature)
        self.user_B_inv = temp_Binv-((temp_Binv).dot(np.outer(temp_armf,temp_armf))).dot(temp_Binv)/(1.+np.dot(temp_armf,(temp_Binv).dot(temp_armf)))
        self.user_y = self.user_y + OBSERVED_REWARD * self.chosen_arm_feature

## GraphUCB.local


In [20]:
class GUCB_local():
    def __init__(self, USER_NUM, ARM_NUM, DIMENSION, DELTA, OUR_GRAPH, OUR_LAPL,CONST_R, CONST_LAMBDA, CONST_V, TIME_HORIZON):
        self.user_num = USER_NUM
        self.arm_num = ARM_NUM
        self.dim = DIMENSION
        self.delta = DELTA
        self.our_graph = OUR_GRAPH
        self.our_lapl = OUR_LAPL
        #self.R = CONST_R
        self.T = TIME_HORIZON
        #self.Lambda = CONST_LAMBDA
        self.v = CONST_V

        self.SELECTED_USER = -1
        self.L=self.our_lapl.copy()+0.01*np.identity(self.user_num)

        self.alpha = CONST_LAMBDA
        self.beta = CONST_V
        self.sigma = CONST_R
        
        self.user_feature_matrix=np.zeros((self.user_num, self.dim))
        self.user_ls=np.zeros((self.user_num, self.dim))

        self.user_v = {u: self.alpha * np.identity(self.dim) for u in range(self.user_num)}
        self.user_avg = {u: np.zeros(self.dim) for u in range(self.user_num)}
        self.user_xx = {u: 0.1 * np.identity(self.dim) for u in range(self.user_num)} 
        self.user_bias = {u: np.zeros(self.dim) for u in range(self.user_num)}
        self.user_counter = {u: 0 for u in range(self.user_num)}
        self.user_h = {u: np.zeros((self.dim, self.dim)) for u in range(self.user_num)}
        self.user_xx_inv = {u: 10 * np.identity(self.dim) for u in range(self.user_num)}
        self.user_v_inv = {u: (1/self.alpha) * np.identity(self.dim) for u in range(self.user_num)}
        self.user_h = {u: np.zeros((self.dim, self.dim)) for u in range(self.user_num)}

    def update_beta(self):
        sum_A = np.zeros((self.dim, self.dim))
        for uu in range(self.user_num):
            sum_A += (self.L[self.SELECTED_USER, uu] **2) * self.user_xx_inv[uu] 

        self.user_h[self.SELECTED_USER] = self.user_xx[self.SELECTED_USER] + (self.alpha**2) * sum_A + 2*self.alpha*self.our_lapl[self.SELECTED_USER, self.SELECTED_USER]*np.identity(self.dim)

        a = np.linalg.det(self.user_v[self.SELECTED_USER])**(1/2)
        b = np.linalg.det(self.alpha*np.identity(self.dim))**(-1/2)
        d = self.sigma*np.sqrt(2*np.log(a*b/self.delta)) 
        self.user_avg[self.SELECTED_USER] = np.dot(self.user_ls.T, self.our_lapl[self.SELECTED_USER])    
        c = np.sqrt(self.alpha)*np.linalg.norm(self.user_avg[self.SELECTED_USER])
        self.beta = d+c
        
    def choose_arm(self, SELECTED_USER, arms_features):
        self.SELECTED_USER=SELECTED_USER
        estimated_payoffs = [0 for j in range(self.arm_num)]
        h_inv = np.linalg.pinv(self.user_h[self.SELECTED_USER]) 

        for j in range(self.arm_num):
            x = arms_features[j]
            x_norm = np.sqrt(np.dot(np.dot(x, h_inv), x))
            mean = np.dot(x, self.user_feature_matrix[self.SELECTED_USER])
            est_y = mean + self.beta * x_norm
            estimated_payoffs[j] = est_y
        
        estimated_payoffs=np.array(estimated_payoffs)
        self.chosen_arm=np.random.choice(np.where(estimated_payoffs == estimated_payoffs.max())[0])       
        self.chosen_arm_feature = arms_features[self.chosen_arm]  
        return self.chosen_arm

    def update_user_features(self, true_payoff):
        x = self.chosen_arm_feature
        user_index = self.SELECTED_USER
        self.user_xx[user_index]+=np.outer(x, x)
        self.user_v[user_index]+=np.outer(x, x)
        self.user_bias[user_index]+=true_payoff*x
        
        self.user_xx_inv[self.SELECTED_USER] = self.user_xx_inv[self.SELECTED_USER] - ((self.user_xx_inv[self.SELECTED_USER]).dot(np.outer(x, x))).dot(self.user_xx_inv[self.SELECTED_USER]) / (1.+np.dot(x,(self.user_xx_inv[self.SELECTED_USER]).dot(x)))
        self.user_v_inv[self.SELECTED_USER] = self.user_v_inv[self.SELECTED_USER] - ((self.user_v_inv[self.SELECTED_USER]).dot(np.outer(x, x))).dot(self.user_v_inv[self.SELECTED_USER]) / (1.+np.dot(x,(self.user_v_inv[self.SELECTED_USER]).dot(x)))    
        self.user_ls[user_index]=np.dot(self.user_xx_inv[self.SELECTED_USER], self.user_bias[user_index])         
        for u in range(self.user_num):
            v_inv=self.user_v_inv[u]
            xx_inv=self.user_xx_inv[u]
            self.user_avg[u]=np.dot(self.user_ls.T, self.L[u])
            self.user_feature_matrix[u]=self.user_ls[u]-self.alpha*np.dot(xx_inv, self.user_avg[u])
            


## CLUB

In [ ]:
import scipy
from scipy.sparse import csgraph, csr_matrix
from scipy.sparse.csgraph import connected_components

class CLUB():
    def __init__(self, USER_NUM, ARM_NUM, DIMENSION, DELTA, CONST_R, CONST_LAMBDA,v, ALPHA_2, TIME_HORIZON):
        self.user_num = USER_NUM
        self.arm_num = ARM_NUM
        self.dim = DIMENSION
        self.delta = DELTA
        self.T = TIME_HORIZON
        self.beta=v
        self.sigma = CONST_R
        self.alpha = CONST_LAMBDA 
        self.alpha_2 = ALPHA_2

        self.served_user_list = []
        self.user_cluster_cov = {i: np.identity(self.dim) for i in range(self.user_num)}
        self.user_cluster_cov_inv = {i: np.identity(self.dim) for i in range(self.user_num)} 
        self.user_feature = np.zeros((self.user_num, self.dim))
        self.covariance = {i: self.alpha*np.identity(self.dim) for i in range(self.user_num)}
        self.covariance_inv = {i: (1/self.alpha)*np.identity(self.dim) for i in range(self.user_num)} 
        self.CBPrime = np.zeros(self.user_num)
        self.user_counters = np.zeros(self.user_num)

        self.adj = np.ones((self.user_num, self.user_num)) 
        self.bias = np.zeros((self.user_num, self.dim))
        self.cluster_num = 0
        self.cluster_list = np.array(list(range(self.user_num)))
        self.user_cluster_bias = np.zeros((self.user_num, self.dim))
        self.user_cluster_feature = np.zeros((self.user_num, self.dim))


    def choose_arm(self, SELECTED_USER, arms_features, CURRENT_TIME):
        self.SELECTED_USER = SELECTED_USER
        user_index = self.SELECTED_USER
        cluster_cov=self.user_cluster_cov[user_index]
        cluster_cov_inv=self.user_cluster_cov_inv[user_index]

        est_payoffs=[]
        for arm in range(0, self.arm_num):
            x = arms_features[arm]
            temp = np.dot(np.dot(x, cluster_cov_inv), x)
            if temp > 0:
                x_norm = np.sqrt(temp)
            else:
                x_norm = np.array(0) 
            est_payoff=np.dot(self.user_cluster_feature[user_index], x)+self.beta*x_norm
            est_payoffs.extend([est_payoff])

        self.chosen_arm = np.argmax(est_payoffs) 
        self.chosen_arm_feature = arms_features[self.chosen_arm]
        return self.chosen_arm
    
    def update_many_things(self, true_payoff):
        selected_item_feature = self.chosen_arm_feature
        user_index = self.SELECTED_USER

        self.covariance[user_index]+=np.outer(selected_item_feature, selected_item_feature)
        self.bias[user_index]+=true_payoff*selected_item_feature
        self.covariance_inv[user_index] = np.linalg.pinv(self.covariance[user_index])
        self.user_feature[user_index]=np.dot(self.covariance_inv[user_index], self.bias[user_index])
        self.CBPrime[user_index]=self.alpha_2*np.sqrt(float(1+np.log(1+self.user_counters[user_index])/float(1+self.user_counters[user_index])))
        self.user_counters[user_index]+=1

        user_index = self.SELECTED_USER
        user_f_diff=np.linalg.norm(self.user_feature[user_index]-self.user_feature, axis=1)
        cb_prime_sum=self.CBPrime[user_index]+self.CBPrime
        ratio=user_f_diff/cb_prime_sum
        big_index=ratio>1.0
        self.adj[big_index, user_index]=0.0
        self.adj[user_index][big_index]=0.0

        self.cluster_num, self.cluster_list = connected_components(csr_matrix(self.adj))

        user_cluster_index=self.cluster_list[user_index]
        all_users_in_the_cluster=list(np.where(self.cluster_list==user_cluster_index)[0])
        self.user_cluster_cov[user_index]=np.identity(self.dim)
        self.user_cluster_bias[user_index]=np.zeros(self.dim)
        for u in all_users_in_the_cluster:
            self.user_cluster_cov[user_index]+=self.covariance[u]-np.identity(self.dim)
            self.user_cluster_bias[user_index]+=self.bias[u]

        self.user_cluster_cov_inv[user_index] = np.linalg.pinv(self.user_cluster_cov[user_index])
        new_cluster_feature=np.dot(self.user_cluster_cov_inv[user_index], self.user_cluster_bias[user_index])

        for u in all_users_in_the_cluster:
            self.user_cluster_feature[u]=new_cluster_feature
 

# Helper class for SCLUB and DyClu

In [13]:
class User():
    def __init__(self, id, theta = None, CoTheta = None):
        self.id = id
        self.theta = theta
class Article():	
	def __init__(self, aid, FV=None):
		self.id = aid
		self.contextFeatureVector = FV

## Generate Graph & Laplacian

In [21]:
def set_graph_and_lapl_for_experiment(user_num, dimension, prob, threshold, our_graph_type, lap_type, gamma):
    synthetic_graph = {}
    synthetic_laplacian = {}
    # graph 1: ER
    random.seed(1234)
    np.random.seed(5678)
    synthetic_graph['ER'] = create_ER_graph(user_num, prob)
    # graph 2: RBF
    np.random.seed(1234)
    synthetic_graph['RBF'] = create_RBF_graph(user_num, dimension, threshold)
    # laplacian
    synthetic_laplacian['ER'] = graph_to_laplacian(synthetic_graph['ER'], lap_type)
    synthetic_laplacian['RBF'] = graph_to_laplacian(synthetic_graph['RBF'], lap_type)

    our_graph = synthetic_graph[our_graph_type]
    our_laplacian = synthetic_laplacian[our_graph_type]
    symm_laplacian = graph_to_laplacian(synthetic_graph[our_graph_type], 'symmetric_normalized')

    users_features = create_users_features(user_num, dimension, our_laplacian, gamma)

    return our_graph, our_laplacian, users_features, symm_laplacian

## Tuning

In [ ]:
def tuning_v_and_lam_for_SELECTED_algo_SYNTH(selected_models,\
                                             v_set, lam_set, user_num, arm_num, dimension, time_horizon, const_R, delta, \
                                             simul_n, epsilon, threshold, prob, gamma, our_graph, our_laplacian, symm_laplacian, \
                                             const_alpha_2, const_tol_1, const_tol_2, nu_type, arm_type):

    #time_cut = time_horizon // 10
    all_models=["Random", "RGraphTS", "LinTS", "SCLUB", "slot4", "CLUB", "DyClu", "SemiRGraphTS", "GraphUCBlocal","IndividualSemiRidgeGraphThompson",\
                "LinTS_Single", "SemiTS_Single"]
    all_models_num = len(all_models)
    not_selected_models = list(set(all_models) - set(selected_models))
    
    results = {algo: [] for algo in all_models}
    Labels = []

    start_cell = time.time()
    for v in v_set:
        for lam in lam_set:
            Labels.append('v='+str(v)+', lambda='+str(lam))  ############## Labels
            cumulated_regret = {algo: [] for algo in all_models}
            total_time = 0

            for simul in range(simul_n):
                computing_time = {n:0 for n in range(0, all_models_num)}
                user_history=np.random.choice(range(user_num), size = time_horizon, replace = True)

                print("<set models>")
                if "RGraphTS" in selected_models:
                    start = time.time()
                    M1 = RidgeGraphThompson(user_num, arm_num, dimension, delta, our_graph, our_laplacian,const_R, lam, v, time_horizon)
                    print(f"RidgeTS, time = {time.time()-start} sec")

                if "LinTS" in selected_models:
                    start = time.time()
                    M2 = IndividualThompson(user_num, arm_num, dimension, delta, const_R, v, epsilon, time_horizon)
                    print(f"IndTS, time = {time.time()-start} sec")

                # M3
                if "SCLUB" in selected_models:
                    start = time.time()
                    M3 = SCLUB(nu=user_num, d=dimension, NoiseScale=v, lambda_=lam)
                    print(f"SCLUB, time = {time.time()-start} sec")
                
                # M5
                if "CLUB" in selected_models:
                    start = time.time()
                    M5 = CLUB(user_num, arm_num, dimension, delta, const_R, lam, v, const_alpha_2, time_horizon)
                    print(f"CLUB, time = {time.time()-start} sec")
                    
                # M6
                if "DyClu" in selected_models:
                    start = time.time()
                    M6 = DyClu(dimension=dimension, lambda_=lam, NoiseScale=v, **dyclu_options)
                    print(f"DyClu, time = {time.time()-start} sec")


                if "SemiRGraphTS" in selected_models:
                    start = time.time()
                    M7 = SemiRidgeGraphThompson(user_num, arm_num, dimension, delta, our_graph, our_laplacian,const_R, lam, v, time_horizon)
                    print(f"SemiRidegeTS, time = {time.time()-start} sec")

                if "GraphUCBlocal" in selected_models:
                    start = time.time()
                    M8 = GUCB_local(user_num, arm_num, dimension, delta, our_graph, our_laplacian,const_R, lam, v, time_horizon)
                    print(f"GUCBlocal, time = {time.time()-start} sec")
                    
                if "IndividualSemiRidgeGraphThompson" in selected_models:
                    start = time.time()
                    M9 = IndividualSemiRidgeGraphThompson(user_num, arm_num, dimension, delta, our_graph, our_laplacian,const_R, lam, v, time_horizon)
                    print(f"IndividualSemiRidgeGraphThompson, time = {time.time()-start} sec")
                    
                if "LinTS_Single" in selected_models:
                    start = time.time()
                    M10 = LinTS_Single(user_num, arm_num, dimension, delta, const_R, v, epsilon, time_horizon)
                    print(f"LinTS_Single, time = {time.time()-start} sec")
                    
                if "SemiTS_Single" in selected_models:
                    start = time.time()
                    M11 = SemiTS_Single(user_num, arm_num, dimension, delta, our_graph, our_laplacian,const_R, lam, v, time_horizon)
                    print(f"SemiTS_Single, time = {time.time()-start} sec")


                print("============================================================")
                RWD0=list()
                RWD1=list()
                RWD2=list()
                RWD3=list()
                #RWD4=list()
                RWD5=list()
                RWD6=list()
                RWD7=list()
                RWD8=list()
                RWD9=list()
                RWD10=list()
                RWD11=list()
                optRWD=list()
                
                initial = time.time()
                for t in range(time_horizon):
                    SELECTED_USER=user_history[t]

                    if arm_type == "normal":
                        arms_features=create_arms_features_normal(arm_num, dimension)
                    else:
                        arms_features=create_arms_features_sparse(arm_num, dimension)

                    errors=np.random.multivariate_normal(np.zeros(arm_num),np.identity(arm_num)*(const_R**2))

                    if nu_type == "0":
                        nu=0
                    else:
                        nu=-np.amax([np.dot(arms_features[i],users_features[SELECTED_USER]) for i in range(arm_num)])

                    start_a_loop = time.time()

                    start_a_loop = time.time()

                    if "Random" in selected_models:
                        #print(t, "RandomSelection", end=" ")
                        start = time.time()
                        arm0=np.random.choice(range(arm_num), replace = True)
                        rwd0=np.dot(arms_features[arm0], users_features[SELECTED_USER])+errors[arm0]+nu
                        RWD0.append(np.dot(arms_features[arm0], users_features[SELECTED_USER]))
                        duration = float(time.time() - start)
                        computing_time[0] += duration
                        #print(f"time = {round(1000*duration, 4)} ms")
                    else:
                        arm0 = -1

                    if "RGraphTS" in selected_models:
                    #print(t, "RidgeGraphThompson", end=" ")
                        start = time.time()
                        arm1=M1.choose_arm(SELECTED_USER, arms_features)
                        rwd1=np.dot(arms_features[arm1], users_features[SELECTED_USER])+errors[arm1]+nu
                        M1.update_B_and_y_and_mu_bar(rwd1)
                        RWD1.append(np.dot(arms_features[arm1], users_features[SELECTED_USER]))
                        duration = float(time.time() - start)
                        computing_time[1] += duration
                    else:
                        arm1 = -1

                    if "LinTS" in selected_models:
                        #print(t, "IndividualThompson", end=" ")
                        start = time.time()
                        arm2=M2.choose_arm(SELECTED_USER, arms_features)
                        rwd2=np.dot(arms_features[arm2], users_features[SELECTED_USER])+errors[arm2]+nu
                        M2.update_B_and_y_and_mu_bar(rwd2)
                        RWD2.append(np.dot(arms_features[arm2], users_features[SELECTED_USER]))
                        duration = float(time.time() - start)
                        computing_time[2] += duration
                        #print(f"time = {round(1000*duration, 4)} ms")
                    else:
                        arm2 = -1

                    if "SCLUB" in selected_models:
                        start = time.time()
                        articlePool = [Article(i, arms_features[i]) for i in range(arm_num)]
                        arm3=M3.decide(articlePool, SELECTED_USER).id
                        rwd3=np.dot(arms_features[arm3], users_features[SELECTED_USER])+errors[arm3]+nu
                        M3.updateParameters(articlePool[arm3], rwd3, SELECTED_USER)
                        RWD3.append(np.dot(arms_features[arm3], users_features[SELECTED_USER]))
                        duration = float(time.time() - start)
                        computing_time[3] += duration
                    else:
                        arm3 = -1

                    arm4 = -1
                    
                    if "CLUB" in selected_models:
                        start = time.time()
                        arm5=M5.choose_arm(SELECTED_USER, arms_features, t)
                        rwd5=np.dot(arms_features[arm5], users_features[SELECTED_USER])+errors[arm5]+nu
                        M5.update_many_things(rwd5)
                        RWD5.append(np.dot(arms_features[arm5], users_features[SELECTED_USER]))
                        duration = float(time.time() - start)
                        computing_time[5] += duration
                    else:
                        arm5 = -1

                    if "DyClu" in selected_models:
                        start = time.time()
                        articlePool = [Article(i, arms_features[i]) for i in range(arm_num)]
                        arm6=M6.decide(articlePool, SELECTED_USER).id
                        rwd6=np.dot(arms_features[arm6], users_features[SELECTED_USER])+errors[arm6]+nu
                        M6.updateParameters(articlePool[arm6], rwd6, SELECTED_USER)
                        RWD6.append(np.dot(arms_features[arm6], users_features[SELECTED_USER]))
                        duration = float(time.time() - start)
                        computing_time[6] += duration
                    else:
                        arm6 = -1

                    if "SemiRGraphTS" in selected_models:
                        #print(t, "SemiRidgeGraphThompson", end=" ")
                        start = time.time()
                        arm7=M7.choose_arm(SELECTED_USER, arms_features)
                        rwd7=np.dot(arms_features[arm7], users_features[SELECTED_USER])+errors[arm7]+nu
                        M7.update_B_and_y_and_mu_bar(rwd7)
                        RWD7.append(np.dot(arms_features[arm7], users_features[SELECTED_USER]))
                        duration = float(time.time() - start)
                        computing_time[7] += duration
                        #print(f"time = {round(1000*duration, 4)} ms")
                    else:
                        arm7 = -1

                    if "GraphUCBlocal" in selected_models:
                        #print(t, "GUCBlocal", end=" ")
                        start = time.time()
                        arm8=M8.choose_arm(SELECTED_USER, arms_features)
                        rwd8=np.dot(arms_features[arm8], users_features[SELECTED_USER])+errors[arm8]+nu
                        M8.update_user_features(rwd8)
                        RWD8.append(np.dot(arms_features[arm8], users_features[SELECTED_USER]))
                        duration = float(time.time() - start)
                        computing_time[8] += duration
                        #print(f"time = {round(1000*duration, 4)} ms")
                    else:
                        arm8 = -1
                        
                    if "IndividualSemiRidgeGraphThompson" in selected_models:
                        #print(t, "IndividualSemiRidgeGraphThompson", end=" ")
                        start = time.time()
                        arm9=M9.choose_arm(SELECTED_USER, arms_features)
                        rwd9=np.dot(arms_features[arm9], users_features[SELECTED_USER])+errors[arm9]+nu
                        M9.update_B_and_y_and_mu_bar(rwd9)
                        RWD9.append(np.dot(arms_features[arm9], users_features[SELECTED_USER]))
                        duration = float(time.time() - start)
                        computing_time[9] += duration
                        #print(f"time = {round(1000*duration, 4)} ms")
                    else:
                        arm9 = -1
                    
                    if "LinTS_Single" in selected_models:
                        #print(t, "LinTS_Single", end=" ")
                        start = time.time()
                        arm10=M10.choose_arm(arms_features)
                        rwd10=np.dot(arms_features[arm10], users_features[SELECTED_USER])+errors[arm10]+nu
                        M10.update_B_and_y_and_mu_bar(rwd10)
                        RWD10.append(np.dot(arms_features[arm10], users_features[SELECTED_USER]))
                        duration = float(time.time() - start)
                        computing_time[10] += duration
                        #print(f"time = {round(1000*duration, 4)} ms")
                    else:
                        arm10 = -1
                        
                    if "SemiTS_Single" in selected_models:
                        #print(t, "SemiTS_Single", end=" ")
                        start = time.time()
                        arm11=M11.choose_arm(SELECTED_USER, arms_features)
                        rwd11=np.dot(arms_features[arm11], users_features[SELECTED_USER])+errors[arm11]+nu
                        M11.update_B_and_y_and_mu_bar(rwd11)
                        RWD11.append(np.dot(arms_features[arm11], users_features[SELECTED_USER]))
                        duration = float(time.time() - start)
                        computing_time[11] += duration
                        #print(f"time = {round(1000*duration, 4)} ms")
                    else:
                        arm11 = -1

                    duration_of_loop = float(time.time() - start_a_loop)
                    true_opt_arm = np.argmax([np.dot(arms_features[i],users_features[SELECTED_USER]) for i in range(arm_num)])
                    
                    if t % 250 == 0:
                        print("(v=",str(v),"lam=", str(lam), ")", t, "user:", SELECTED_USER, "/ selected arms:", arm0, arm1, arm2, arm3, arm4, arm5, arm6, arm7, arm8, arm9, arm10, arm11, "/ opt_arm:", true_opt_arm, "/ time :", round(duration_of_loop, 3), "sec")
                        print("=====")
                    optRWD.append(np.amax([np.dot(arms_features[i],users_features[SELECTED_USER]) for i in range(arm_num)]))

                time_for_one_iter = float(time.time() - initial)
                total_time += time_for_one_iter
                print(f"iteration = {simul+1}/{simul_n}, time = {round(time_for_one_iter, 3)} sec")
                print("==========")
                time.sleep(1)

                if "Random" in selected_models:
                    cumulated_regret["Random"].append(np.cumsum(optRWD)-np.cumsum(RWD0))
                if "RGraphTS" in selected_models:
                    cumulated_regret["RGraphTS"].append(np.cumsum(optRWD)-np.cumsum(RWD1))
                if "LinTS" in selected_models:
                    cumulated_regret["LinTS"].append(np.cumsum(optRWD)-np.cumsum(RWD2))
                if "SCLUB" in selected_models:
                    cumulated_regret["SCLUB"].append(np.cumsum(optRWD)-np.cumsum(RWD3))
                # slot4
                if "CLUB" in selected_models:
                    cumulated_regret["CLUB"].append(np.cumsum(optRWD)-np.cumsum(RWD5)) 
                if "DyClu" in selected_models:
                    cumulated_regret["DyClu"].append(np.cumsum(optRWD)-np.cumsum(RWD6))
                if "SemiRGraphTS" in selected_models:
                    cumulated_regret["SemiRGraphTS"].append(np.cumsum(optRWD)-np.cumsum(RWD7))
                if "GraphUCBlocal" in selected_models:
                    cumulated_regret["GraphUCBlocal"].append(np.cumsum(optRWD)-np.cumsum(RWD8))
                if "IndividualSemiRidgeGraphThompson" in selected_models:
                    cumulated_regret["IndividualSemiRidgeGraphThompson"].append(np.cumsum(optRWD)-np.cumsum(RWD9))
                if "LinTS_Single" in selected_models:
                    cumulated_regret["LinTS_Single"].append(np.cumsum(optRWD)-np.cumsum(RWD10))
                if "SemiTS_Single" in selected_models:
                    cumulated_regret["SemiTS_Single"].append(np.cumsum(optRWD)-np.cumsum(RWD11))
                
                for MODEL in not_selected_models:
                    cumulated_regret[MODEL].append(np.zeros(time_horizon))

                #----------
                info_computing_time =""
                for i in range(0, all_models_num):
                    temp = float(1000*computing_time[i]/time_horizon)
                    info_computing_time = info_computing_time + all_models[i] +": "+ str(round(temp, 4)) +" ms\n"
                info_computing_time = info_computing_time + "=========================\nTOTAL TIME: "+str(round(total_time, 3))+" sec"

                print(info_computing_time)
                print(f"v={v} / lam={lam}")
                time.sleep(1)

            for MODEL in all_models:
                results[MODEL].append(cumulated_regret[MODEL])
                
    print("\n###############\n")
    print("n =", user_num, "/ arms =", arm_num, "/ d =", dimension, "/ T =", time_horizon)
    print(f"TOTAL  TIME in THIS CELL: {round(time.time() - start_cell, 3)} sec")
    
    return Labels, results

### Check the tuning results

In [24]:
def show_tuning_results(MODEL, v_and_lam_num, colors, Labels, results):
    RT_output = []
    plt.figure(figsize = (8,8))
    for i in range(v_and_lam_num):
        regrets = np.array(results[MODEL][i])
        med_regrets = np.median(regrets, axis=0)
        RT_output.append(med_regrets[-1])
        T = med_regrets.shape[0]
        plt.plot(np.arange(1,T+1), med_regrets, label=Labels[i], color=colors[i])
        plt.title(MODEL)
    plt.legend()
    plt.ylabel('Regrets')
    plt.show()      
    return RT_output

## Choose best hyper parameters (by tuning results)

In [ ]:
def return_best_v_and_lambda_pair_real(selected_models, 
                                       user_num, arm_num, dimension, time_horizon, arm_type, colors, results, RT_dict):

    all_models=["Random", "RGraphTS", "LinTS", "SCLUB", "slot4", "CLUB", "DyClu", "SemiRGraphTS", "GraphUCBlocal","IndividualSemiRidgeGraphThompson",\
                "LinTS_Single", "SemiTS_Single"]
    index_dict={ MODEL:RT_dict[MODEL].index(min(RT_dict[MODEL])) for MODEL in selected_models}

    name = "n=" + str(user_num) + ",arms="+str(arm_num)+",d="+str(dimension)+",T="+str(time_horizon)+",arm_type="+arm_type
    tuning_info = "#"+name+"\n"
    tuning_info += "best_v_lam_pair_dict = {\n"
    best_v_lam_pair_dict = {}

    plt.figure(figsize=(8,8))
    regrets = np.zeros(1) 
    for i, model in enumerate(all_models):

        if model not in selected_models:
            best_v_lam_pair_dict[model] = {"v": 1.1, "lam": 1.1}
            continue

        v_and_lam_index = index_dict[model]
        v = v_set[v_and_lam_index // lam_num]
        lam = lam_set[v_and_lam_index % lam_num]
        
        best_v_lam_pair_dict[model] = {"v": v, "lam": lam}           

        regrets = np.array(results[model][index_dict[model]])
        med_regrets = np.median(regrets, axis=0)
        T = med_regrets.shape[0]

        if model == "Random":
            time =  T # T // 5
            plt.plot(np.arange(1,time+1), med_regrets[:time], label=model, color=colors[i])
        else:
            plt.plot(np.arange(1,T+1), med_regrets, label=model, color=colors[i])
        plt.title('Tuning Results (one simul, small T)')

    plt.legend()
    plt.ylabel('Regrets')
    plt.legend(loc='center', bbox_to_anchor=(0.5,-0.1),fancybox=True,ncol=4)
                      
    tuning_info = "#"+name+"\n=====================\n"
    for key,value  in best_v_lam_pair_dict.items():
        temp = key +": v= "+ str(value["v"]) +", lam = "+str(value["lam"])+"\n"
        tuning_info += temp
                      
    plt.figtext(0.5, -0.4, tuning_info, ha="center", fontsize=10)
    print(tuning_info)
    plt.show()

    return best_v_lam_pair_dict, tuning_info

## Plot (using best v and lam)

In [ ]:
def run_all_algo_with_best_v_lam_SYNTH(selected_models, \
                                       best_v_lam_pair_dict, user_num, arm_num,  dimension, time_horizon, const_R, delta, simul_n, epsilon, threshold, prob, gamma, our_graph, our_laplacian, symm_laplacian, const_alpha_2, const_tol_1, const_tol_2, nu_type, arm_type):
    time_cut = time_horizon // 50 
    v_lam = best_v_lam_pair_dict

    all_models=["Random", "RGraphTS", "LinTS", "SCLUB", "slot4", "CLUB", "DyClu", "SemiRGraphTS", "GraphUCBlocal","IndividualSemiRidgeGraphThompson",\
                "LinTS_Single", "SemiTS_Single"]
    all_models_num = len(all_models)
    not_selected_models = list(set(all_models) - set(selected_models))
    
    cumulated_regret = {algo: [] for algo in all_models}

    total_time = 0
    for simul in range(simul_n): 
       
        computing_time = {n:0 for n in range(0, all_models_num)}
        user_history=np.random.choice(range(user_num), size = time_horizon, replace = True)

        print("<set models>")
        if "RGraphTS" in selected_models:
            start = time.time()
            M1 = RidgeGraphThompson(user_num, arm_num, dimension, delta, our_graph, our_laplacian,const_R, v_lam["RGraphTS"]["lam"], v_lam["RGraphTS"]["v"], time_horizon)
            print(f"RidgeTS, time = {time.time()-start} sec")
        
        if "LinTS" in selected_models:
            start = time.time()
            M2 = IndividualThompson(user_num, arm_num, dimension, delta, const_R, v_lam["LinTS"]["v"], epsilon, time_horizon)
            print(f"IndTS, time = {time.time()-start} sec")

        # M3
        if "SCLUB" in selected_models:
            start = time.time()
            M3 = SCLUB(nu=user_num, d=dimension, NoiseScale=v_lam["SCLUB"]["v"], lambda_=v_lam["SCLUB"]["lam"])
            print(f"SCLUB, time = {time.time()-start} sec")

        # M5
        if "CLUB" in selected_models:
            start = time.time()
            M5 = CLUB(user_num, arm_num, dimension, delta, const_R, v_lam["CLUB"]["lam"], v_lam["CLUB"]["v"], const_alpha_2, time_horizon)
            print(f"CLUB, time = {time.time()-start} sec")

        # M6
        if "DyClu" in selected_models:
            start = time.time()
            M6 = DyClu(dimension=dimension, lambda_=v_lam["DyClu"]["lam"], NoiseScale=v_lam["DyClu"]["v"], **dyclu_options)
            print(f"DyClu, time = {time.time()-start} sec")


        if "SemiRGraphTS" in selected_models:
            start = time.time()
            M7 = SemiRidgeGraphThompson(user_num, arm_num, dimension, delta, our_graph, our_laplacian,const_R, v_lam["SemiRGraphTS"]["lam"], v_lam["SemiRGraphTS"]["v"], time_horizon)
            print(f"SemiRidegeTS, time = {time.time()-start} sec")
        
        if "GraphUCBlocal" in selected_models:
            start = time.time()
            M8 = GUCB_local(user_num, arm_num, dimension, delta, our_graph, our_laplacian,const_R, v_lam["GraphUCBlocal"]["lam"], v_lam["GraphUCBlocal"]["v"], time_horizon)
            print(f"GUCBlocal, time = {time.time()-start} sec")
            
        if "IndividualSemiRidgeGraphThompson" in selected_models:
            start = time.time()
            M9 = IndividualSemiRidgeGraphThompson(user_num, arm_num, dimension, delta, our_graph, our_laplacian,const_R, v_lam["IndividualSemiRidgeGraphThompson"]["lam"], v_lam["IndividualSemiRidgeGraphThompson"]["v"], time_horizon)
            print(f"IndividualSemiRidgeGraphThompson, time = {time.time()-start} sec")
            
        if "LinTS_Single" in selected_models:
            start = time.time()
            M10 = LinTS_Single(user_num, arm_num, dimension, delta, const_R, v_lam["LinTS_Single"]["v"], epsilon, time_horizon)
            print(f"LinTS_Single, time = {time.time()-start} sec")
            
        if "SemiTS_Single" in selected_models:
            start = time.time()
            M11 = SemiTS_Single(user_num, arm_num, dimension, delta, our_graph, our_laplacian,const_R, v_lam["SemiTS_Single"]["lam"], v_lam["SemiTS_Single"]["v"], time_horizon)
            print(f"SemiTS_Single, time = {time.time()-start} sec")

        print("============================================================")
        RWD0=list()
        RWD1=list()
        RWD2=list()
        RWD3=list()
        #RWD4=list()
        RWD5=list()
        RWD6=list()
        RWD7=list()
        RWD8=list()
        RWD9=list()
        RWD10=list()
        RWD11=list()
        optRWD=list()
        
        initial = time.time()
        
        for t in range(time_horizon):

            SELECTED_USER=user_history[t]

            if arm_type == "normal":
                arms_features=create_arms_features_normal(arm_num, dimension)
            else:
                arms_features=create_arms_features_sparse(arm_num, dimension)

            errors=np.random.multivariate_normal(np.zeros(arm_num),np.identity(arm_num)*(const_R**2))

            if nu_type == "0":
                nu=0
            else:
                nu=-np.amax([np.dot(arms_features[i],users_features[SELECTED_USER]) for i in range(arm_num)])

            start_a_loop = time.time()

            if "Random" in selected_models:
                #print(t, "RandomSelection", end=" ")
                start = time.time()
                arm0=np.random.choice(range(arm_num), replace = True)
                rwd0=np.dot(arms_features[arm0], users_features[SELECTED_USER])+errors[arm0]+nu
                RWD0.append(np.dot(arms_features[arm0], users_features[SELECTED_USER]))
                duration = float(time.time() - start)
                computing_time[0] += duration
                #print(f"time = {round(1000*duration, 4)} ms")
            else:
                arm0 = -1

            if "RGraphTS" in selected_models:
            #print(t, "RidgeGraphThompson", end=" ")
                start = time.time()
                arm1=M1.choose_arm(SELECTED_USER, arms_features)
                rwd1=np.dot(arms_features[arm1], users_features[SELECTED_USER])+errors[arm1]+nu
                M1.update_B_and_y_and_mu_bar(rwd1)
                RWD1.append(np.dot(arms_features[arm1], users_features[SELECTED_USER]))
                duration = float(time.time() - start)
                computing_time[1] += duration
            else:
                arm1 = -1

            if "LinTS" in selected_models:
                #print(t, "IndividualThompson", end=" ")
                start = time.time()
                arm2=M2.choose_arm(SELECTED_USER, arms_features)
                rwd2=np.dot(arms_features[arm2], users_features[SELECTED_USER])+errors[arm2]+nu
                M2.update_B_and_y_and_mu_bar(rwd2)
                RWD2.append(np.dot(arms_features[arm2], users_features[SELECTED_USER]))
                duration = float(time.time() - start)
                computing_time[2] += duration
                #print(f"time = {round(1000*duration, 4)} ms")
            else:
                arm2 = -1

            if "SCLUB" in selected_models:
                start = time.time()
                articlePool = [Article(i, arms_features[i]) for i in range(arm_num)]
                arm3=M3.decide(articlePool, SELECTED_USER).id
                rwd3=np.dot(arms_features[arm3], users_features[SELECTED_USER])+errors[arm3]+nu
                M3.updateParameters(articlePool[arm3], rwd3, SELECTED_USER)
                RWD3.append(np.dot(arms_features[arm3], users_features[SELECTED_USER]))
                duration = float(time.time() - start)
                computing_time[3] += duration
            else:
                arm3 = -1

            arm4 = -1

            if "CLUB" in selected_models:
                start = time.time()
                arm5=M5.choose_arm(SELECTED_USER, arms_features, t)
                rwd5=np.dot(arms_features[arm5], users_features[SELECTED_USER])+errors[arm5]+nu
                M5.update_many_things(rwd5)
                RWD5.append(np.dot(arms_features[arm5], users_features[SELECTED_USER]))
                duration = float(time.time() - start)
                computing_time[5] += duration
            else:
                arm5 = -1

            if "DyClu" in selected_models:
                start = time.time()
                articlePool = [Article(i, arms_features[i]) for i in range(arm_num)]
                arm6=M6.decide(articlePool, SELECTED_USER).id
                rwd6=np.dot(arms_features[arm6], users_features[SELECTED_USER])+errors[arm6]+nu
                M6.updateParameters(articlePool[arm6], rwd6, SELECTED_USER)
                RWD6.append(np.dot(arms_features[arm6], users_features[SELECTED_USER]))
                duration = float(time.time() - start)
                computing_time[6] += duration
            else:
                arm6 = -1

            if "SemiRGraphTS" in selected_models:
                #print(t, "SemiRidgeGraphThompson", end=" ")
                start = time.time()
                arm7=M7.choose_arm(SELECTED_USER, arms_features)
                rwd7=np.dot(arms_features[arm7], users_features[SELECTED_USER])+errors[arm7]+nu
                M7.update_B_and_y_and_mu_bar(rwd7)
                RWD7.append(np.dot(arms_features[arm7], users_features[SELECTED_USER]))
                duration = float(time.time() - start)
                computing_time[7] += duration
                #print(f"time = {round(1000*duration, 4)} ms")
            else:
                arm7 = -1

            if "GraphUCBlocal" in selected_models:
                #print(t, "GUCBlocal", end=" ")
                start = time.time()
                arm8=M8.choose_arm(SELECTED_USER, arms_features)
                rwd8=np.dot(arms_features[arm8], users_features[SELECTED_USER])+errors[arm8]+nu
                M8.update_user_features(rwd8)
                RWD8.append(np.dot(arms_features[arm8], users_features[SELECTED_USER]))
                duration = float(time.time() - start)
                computing_time[8] += duration
                #print(f"time = {round(1000*duration, 4)} ms")
            else:
                arm8 = -1

            if "IndividualSemiRidgeGraphThompson" in selected_models:
                #print(t, "IndividualSemiRidgeGraphThompson", end=" ")
                start = time.time()
                arm9=M9.choose_arm(SELECTED_USER, arms_features)
                rwd9=np.dot(arms_features[arm9], users_features[SELECTED_USER])+errors[arm9]+nu
                M9.update_B_and_y_and_mu_bar(rwd9)
                RWD9.append(np.dot(arms_features[arm9], users_features[SELECTED_USER]))
                duration = float(time.time() - start)
                computing_time[9] += duration
                #print(f"time = {round(1000*duration, 4)} ms")
            else:
                arm9 = -1
                
            if "LinTS_Single" in selected_models:
                #print(t, "LinTS_Single", end=" ")
                start = time.time()
                arm10=M10.choose_arm(arms_features)
                rwd10=np.dot(arms_features[arm10], users_features[SELECTED_USER])+errors[arm10]+nu
                M10.update_B_and_y_and_mu_bar(rwd10)
                RWD10.append(np.dot(arms_features[arm10], users_features[SELECTED_USER]))
                duration = float(time.time() - start)
                computing_time[10] += duration
                #print(f"time = {round(1000*duration, 4)} ms")
            else:
                arm10 = -1
                
            if "SemiTS_Single" in selected_models:
                #print(t, "SemiTS_Single", end=" ")
                start = time.time()
                arm11=M11.choose_arm(SELECTED_USER, arms_features)
                rwd11=np.dot(arms_features[arm11], users_features[SELECTED_USER])+errors[arm11]+nu
                M11.update_B_and_y_and_mu_bar(rwd11)
                RWD11.append(np.dot(arms_features[arm11], users_features[SELECTED_USER]))
                duration = float(time.time() - start)
                computing_time[11] += duration
                #print(f"time = {round(1000*duration, 4)} ms")
            else:
                arm11 = -1

            duration_of_loop = float(time.time() - start_a_loop)
            true_opt_arm = np.argmax([np.dot(arms_features[i],users_features[SELECTED_USER]) for i in range(arm_num)])
            
            if t % 500 == 0:
                print(t, "user:", SELECTED_USER, "/ selected arms:", arm0, arm1, arm2, arm3, arm4, arm5, arm6, arm7, arm8, arm9, arm10, arm11,"/ opt_arm:", true_opt_arm, "/ time :", round(duration_of_loop, 3), "sec")
                print("=====")
                
            optRWD.append(np.amax([np.dot(arms_features[i],users_features[SELECTED_USER]) for i in range(arm_num)]))
            
        time_for_one_iter = float(time.time() - initial)
        total_time += time_for_one_iter
        print(f"iteration = {simul+1}/{simul_n}, time = {round(time_for_one_iter, 3)} sec")
        print("==========")
        time.sleep(1)

        if "Random" in selected_models:
            cumulated_regret["Random"].append(np.cumsum(optRWD)-np.cumsum(RWD0))
        if "RGraphTS" in selected_models:
            cumulated_regret["RGraphTS"].append(np.cumsum(optRWD)-np.cumsum(RWD1))
        if "LinTS" in selected_models:
            cumulated_regret["LinTS"].append(np.cumsum(optRWD)-np.cumsum(RWD2))
        if "SCLUB" in selected_models:
            cumulated_regret["SCLUB"].append(np.cumsum(optRWD)-np.cumsum(RWD3))
        # slot4
        if "CLUB" in selected_models:
            cumulated_regret["CLUB"].append(np.cumsum(optRWD)-np.cumsum(RWD5)) 
        if "DyClu" in selected_models:
            cumulated_regret["DyClu"].append(np.cumsum(optRWD)-np.cumsum(RWD6))
        if "SemiRGraphTS" in selected_models:
            cumulated_regret["SemiRGraphTS"].append(np.cumsum(optRWD)-np.cumsum(RWD7))
        if "GraphUCBlocal" in selected_models:
            cumulated_regret["GraphUCBlocal"].append(np.cumsum(optRWD)-np.cumsum(RWD8))
        if "IndividualSemiRidgeGraphThompson" in selected_models:
            cumulated_regret["IndividualSemiRidgeGraphThompson"].append(np.cumsum(optRWD)-np.cumsum(RWD9))
        if "LinTS_Single" in selected_models:
            cumulated_regret["LinTS_Single"].append(np.cumsum(optRWD)-np.cumsum(RWD10))
        if "SemiTS_Single" in selected_models:
            cumulated_regret["SemiTS_Single"].append(np.cumsum(optRWD)-np.cumsum(RWD11))
            
        for MODEL in not_selected_models:
            cumulated_regret[MODEL].append(np.zeros(time_horizon))

        #----------
    info_computing_time =""
    for i in range(0, all_models_num):
        temp = float(1000*computing_time[i]/time_horizon)
        info_computing_time = info_computing_time + all_models[i] +": "+ str(round(temp, 4)) +" ms\n"
    info_computing_time = info_computing_time + "=========================\nTOTAL TIME: "+str(round(total_time, 4))+" sec"

    print(info_computing_time) 

    return cumulated_regret, info_computing_time

In [15]:
def show_and_save_plot(SAVE, const_linewidth, const_transparency, const_cut_random, const_cut_semi, cumulated_regret_dict, user_num, arm_num,  dimension, time_horizon, const_R, delta, simul_n, epsilon, threshold, prob, gamma, lap_type, our_graph_type, const_alpha_2, const_tol_1, const_tol_2, nu_type, arm_type, info_computing_time, tuning_info, IMG_PATH, NOW):
    cut_random = const_cut_random
    cut_semi = const_cut_semi
    
    steps=np.arange(1,time_horizon+1)
    LW = const_linewidth
    TP = const_transparency
    FTP = 0.2

    plt.figure(figsize=(7,7))
    plt.plot(steps[:time_horizon//cut_random],np.median(cumulated_regret_dict["Random"],axis=0)[:time_horizon//cut_random],'y',label='Random')
    lower = np.percentile(cumulated_regret_dict["Random"],25,axis=0)[:time_horizon//cut_random]
    upper = np.percentile(cumulated_regret_dict["Random"],75,axis=0)[:time_horizon//cut_random]
    plt.fill_between(steps[:time_horizon//cut_random], lower, upper, color = 'y', alpha=FTP)
    #plt.plot(steps,np.median(cumulated_regret_Random,axis=0),'orange',linewidth = LW, alpha =  TP, label='Random Selection')

    plt.plot(steps,np.median(cumulated_regret_dict["RGraphTS"],axis=0),'r', linewidth = LW, alpha =  TP, label='RGraphTS')
    lower = np.percentile(cumulated_regret_dict["RGraphTS"],25,axis=0)
    upper = np.percentile(cumulated_regret_dict["RGraphTS"],75,axis=0)
    plt.fill_between(steps, lower, upper, color = 'r', alpha=FTP)
    
    plt.plot(steps,np.median(cumulated_regret_dict["LinTS"],axis=0),'g', linewidth = LW, alpha =  TP, label='LinTS')
    lower = np.percentile(cumulated_regret_dict["LinTS"],25,axis=0)
    upper = np.percentile(cumulated_regret_dict["LinTS"],75,axis=0)
    plt.fill_between(steps, lower, upper, color = 'g', alpha=FTP)
    
    plt.plot(steps,np.median(cumulated_regret_dict["SCLUB"],axis=0),'r', linewidth = LW, alpha =  TP, label='CLUB')
    lower = np.percentile(cumulated_regret_dict["SCLUB"],25,axis=0)
    upper = np.percentile(cumulated_regret_dict["SCLUB"],75,axis=0)
    plt.fill_between(steps, lower, upper, color = 'r', alpha=FTP)
    # slot4
    
    plt.plot(steps,np.median(cumulated_regret_dict["CLUB"],axis=0),'slategrey', linewidth = LW, alpha =  TP, label='CLUB')
    lower = np.percentile(cumulated_regret_dict["CLUB"],25,axis=0)
    upper = np.percentile(cumulated_regret_dict["CLUB"],75,axis=0)
    plt.fill_between(steps, lower, upper, color = 'slategrey', alpha=FTP)
    
    plt.plot(steps,np.median(cumulated_regret_dict["DyClu"],axis=0),'aqua', linewidth = LW, alpha =  TP, label='CLUB')
    lower = np.percentile(cumulated_regret_dict["DyClu"],25,axis=0)
    upper = np.percentile(cumulated_regret_dict["DyClu"],75,axis=0)
    plt.fill_between(steps, lower, upper, color = 'aqua', alpha=FTP)

    
    plt.plot(steps[:time_horizon//cut_semi],np.median(cumulated_regret_dict["SemiRGraphTS"],axis=0)[:time_horizon//cut_semi],'blue', linewidth = LW, alpha =  TP, label='SemiRGraphTS')
    lower = np.percentile(cumulated_regret_dict["SemiRGraphTS"],25,axis=0)[:time_horizon//cut_semi]
    upper = np.percentile(cumulated_regret_dict["SemiRGraphTS"],75,axis=0)[:time_horizon//cut_semi]
    plt.fill_between(steps[:time_horizon//cut_semi], lower, upper, color = 'blue', alpha=FTP)
    #plt.plot(steps,np.median(cumulated_regret_dict["SemiRGraphTS"],axis=0),'pink', linewidth = LW, alpha =  TP, label='SemiRGraphTS')
    
    #plt.plot(steps[:time_horizon//cut_alt],np.median(cumulated_regret_dict["GraphUCBlocal"],axis=0)[:time_horizon//cut_alt],'brown', linewidth = LW, alpha =  TP, label='GraphUCBlocal')
    plt.plot(steps,np.median(cumulated_regret_dict["GraphUCBlocal"],axis=0),'brown', linewidth = LW, alpha =  TP, label='GraphUCBlocal')
    lower = np.percentile(cumulated_regret_dict["GraphUCBlocal"],25,axis=0)
    upper = np.percentile(cumulated_regret_dict["GraphUCBlocal"],75,axis=0)
    plt.fill_between(steps, lower, upper, color = 'brown', alpha=FTP)
    
    plt.plot(steps[:time_horizon//cut_semi],np.median(cumulated_regret_dict["IndividualSemiRidgeGraphThompson"],axis=0)[:time_horizon//cut_semi],'pink', linewidth = LW, alpha =  TP, label='IndividualSemiRidgeGraphThompson')
    lower = np.percentile(cumulated_regret_dict["IndividualSemiRidgeGraphThompson"],25,axis=0)[:time_horizon//cut_semi]
    upper = np.percentile(cumulated_regret_dict["IndividualSemiRidgeGraphThompson"],75,axis=0)[:time_horizon//cut_semi]
    plt.fill_between(steps[:time_horizon//cut_semi], lower, upper, color = 'pink', alpha=FTP)
    
    plt.plot(steps[:time_horizon//cut_semi],np.median(cumulated_regret_dict["LinTS_Single"],axis=0)[:time_horizon//cut_semi],'lime', linewidth = LW, alpha =  TP, label='LinTS_Single')
    lower = np.percentile(cumulated_regret_dict["LinTS_Single"],25,axis=0)[:time_horizon//cut_semi]
    upper = np.percentile(cumulated_regret_dict["LinTS_Single"],75,axis=0)[:time_horizon//cut_semi]
    plt.fill_between(steps[:time_horizon//cut_semi], lower, upper, color = 'lime', alpha=FTP)
    
    plt.plot(steps[:time_horizon//cut_semi],np.median(cumulated_regret_dict["SemiTS_Single"],axis=0)[:time_horizon//cut_semi],'m', linewidth = LW, alpha =  TP, label='SemiTS_Single')
    lower = np.percentile(cumulated_regret_dict["SemiTS_Single"],25,axis=0)[:time_horizon//cut_semi]
    upper = np.percentile(cumulated_regret_dict["SemiTS_Single"],75,axis=0)[:time_horizon//cut_semi]
    plt.fill_between(steps[:time_horizon//cut_semi], lower, upper, color = 'm', alpha=FTP)
    
    plt.xlabel('Decision Point')
    plt.ylabel('Cumulative Regret')
    title_one = "users="+str(user_num)+", arms="+str(arm_num)+", dim="+str(dimension)+", T="+str(time_horizon) +", gamma="+str(gamma)
    title_two = "nu_type="+nu_type +", arm_type="+arm_type+", simul="+str(simul_n)
    title_four = "tol_1="+str(round(const_tol_1,6))+", tol_2="+str(round(const_tol_2, 6))
    title = NOW + "\n------------------------------\n"+title_one+"\n"+title_two+"\n"+title_four
    plt.title(title+"\n")

    plt.legend(loc='center', bbox_to_anchor=(0.5,-0.15),fancybox=True,ncol=4)
    plt.figtext(0.5, -0.4, info_computing_time, ha="center", fontsize=10)
    plt.figtext(0.5, -0.8, tuning_info, ha="center", fontsize=10)

    if SAVE == True:
        plt.savefig(IMG_PATH, bbox_inches='tight')
    plt.show()

    print(f"\n<parameters>\nuser_num = {user_num} / arm_num = {arm_num} / dimension = {dimension}")
    print(f"delta = {delta} / R = {const_R} / gamma (for smooth Theta) = {gamma}")
    print("-----")
    print(f"const_tol_1 = {const_tol_1} / const_tol_2 = {const_tol_2}")
    print(f"graph = {our_graph_type} / laplacian={lap_type} / threshold = {threshold} / prob = {prob} / const_alpha_2 = {const_alpha_2}")
    print("########################################")
    print("\n"+"<mean computing time>\n"+info_computing_time)
    print("########################################")
    print("\n"+"<best tuning parameter>\n"+tuning_info)

# Runtime

In [ ]:
def RUNTIME_for_selected_model_SYNTH(selected_models, \
                                     best_v_lam_pair_dict, user_num, arm_num,  dimension, time_horizon, const_R, delta, simul_n, epsilon, threshold, prob, gamma, our_graph, our_laplacian, symm_laplacian, const_alpha_2, const_tol_1, const_tol_2, nu_type, arm_type):
    time_cut = time_horizon // 10 
    v_lam = best_v_lam_pair_dict

    all_models=["Random", "RGraphTS", "LinTS", "SCLUB", "slot4", "CLUB", "DyClu", "SemiRGraphTS", "GraphUCBlocal","IndividualSemiRidgeGraphThompson",\
                "LinTS_Single", "SemiTS_Single"]
    all_models_num = len(all_models)
    not_selected_models = list(set(all_models) - set(selected_models))
    
    cumulated_regret = {algo: [] for algo in all_models}
    
    appropriate_n_d = (user_num * dimension < 5000) # for large n*d, we will not execute GobLin and HOB

    total_time = 0
    for simul in range(simul_n): 
       
        computing_time = {n:0 for n in range(0, all_models_num)}
        record_runtime = {MODEL : [] for MODEL in all_models}
        user_history=np.random.choice(range(user_num), size = time_horizon, replace = True)

        print("<set models>")
        if "RGraphTS" in selected_models:
            start = time.time()
            M1 = RidgeGraphThompson(user_num, arm_num, dimension, delta, our_graph, our_laplacian,const_R, v_lam["RGraphTS"]["lam"], v_lam["RGraphTS"]["v"], time_horizon)
            print(f"RidgeTS, time = {time.time()-start} sec")
        
        if "LinTS" in selected_models:
            start = time.time()
            M2 = IndividualThompson(user_num, arm_num, dimension, delta, const_R, v_lam["LinTS"]["v"], epsilon, time_horizon)
            print(f"IndTS, time = {time.time()-start} sec")
            
        if "SCLUB" in selected_models:
            start = time.time()
            M3 = SCLUB(nu=user_num, d=dimension, NoiseScale=v_lam["SCLUB"]["v"], lambda_=v_lam["SCLUB"]["lam"])
            print(f"SCLUB, time = {time.time()-start} sec")

        # M5
        if "CLUB" in selected_models:
            start = time.time()
            M5 = CLUB(user_num, arm_num, dimension, delta, const_R, v_lam["CLUB"]["lam"], v_lam["CLUB"]["v"], const_alpha_2, time_horizon)
            print(f"CLUB, time = {time.time()-start} sec")

        # M6
        if "DyClu" in selected_models:
            start = time.time()
            M6 = DyClu(dimension=dimension, lambda_=v_lam["DyClu"]["lam"], NoiseScale=v_lam["DyClu"]["v"], **dyclu_options)
            print(f"DyClu, time = {time.time()-start} sec")

        if "SemiRGraphTS" in selected_models:
            start = time.time()
            M7 = SemiRidgeGraphThompson(user_num, arm_num, dimension, delta, our_graph, our_laplacian,const_R, v_lam["SemiRGraphTS"]["lam"], v_lam["SemiRGraphTS"]["v"], time_horizon)
            print(f"SemiRidegeTS, time = {time.time()-start} sec")
        
        if "GraphUCBlocal" in selected_models:
            start = time.time()
            M8 = GUCB_local(user_num, arm_num, dimension, delta, our_graph, our_laplacian,const_R, v_lam["GraphUCBlocal"]["lam"], v_lam["GraphUCBlocal"]["v"], time_horizon)
            print(f"GUCBlocal, time = {time.time()-start} sec")
            
        if "IndividualSemiRidgeGraphThompson" in selected_models:
            start = time.time()
            M9 = IndividualSemiRidgeGraphThompson(user_num, arm_num, dimension, delta, our_graph, our_laplacian,const_R, v_lam["IndividualSemiRidgeGraphThompson"]["lam"], v_lam["IndividualSemiRidgeGraphThompson"]["v"], time_horizon)
            print(f"IndividualSemiRidgeGraphThompson, time = {time.time()-start} sec")
            
        if "LinTS_Single" in selected_models:
            start = time.time()
            M10 = LinTS_Single(user_num, arm_num, dimension, delta, const_R, v_lam["LinTS_Single"]["v"], epsilon, time_horizon)
            print(f"LinTS_Single, time = {time.time()-start} sec")
            
        if "SemiTS_Single" in selected_models:
            start = time.time()
            M11 = SemiTS_Single(user_num, arm_num, dimension, delta, our_graph, our_laplacian,const_R, v_lam["SemiTS_Single"]["lam"], v_lam["SemiTS_Single"]["v"], time_horizon)
            print(f"SemiTS_Single, time = {time.time()-start} sec")

        print("============================================================")
        RWD0=list()
        RWD1=list()
        RWD2=list()
        RWD3=list()
        #RWD4=list()
        RWD5=list()
        RWD6=list()
        RWD7=list()
        RWD8=list()
        RWD9=list()
        RWD10=list()
        RWD11=list()
        optRWD=list()
        
        initial = time.time()
        
        for t in range(time_horizon):

            SELECTED_USER=user_history[t]

            if arm_type == "normal":
                arms_features=create_arms_features_normal(arm_num, dimension)
            else:
                arms_features=create_arms_features_sparse(arm_num, dimension)

            errors=np.random.multivariate_normal(np.zeros(arm_num),np.identity(arm_num)*(const_R**2))

            if nu_type == "0":
                nu=0
            else:
                nu=-np.amax([np.dot(arms_features[i],users_features[SELECTED_USER]) for i in range(arm_num)])

            start_a_loop = time.time()

            if "Random" in selected_models:
                #print(t, "RandomSelection", end=" ")
                start = time.time()
                arm0=np.random.choice(range(arm_num), replace = True)
                rwd0=np.dot(arms_features[arm0], users_features[SELECTED_USER])+errors[arm0]+nu
                RWD0.append(np.dot(arms_features[arm0], users_features[SELECTED_USER]))
                duration = float(time.time() - start)
                computing_time[0] += duration
                record_runtime["Random"].append(duration)
                #print(f"time = {round(1000*duration, 4)} ms")
            else:
                arm0 = -1

            if "RGraphTS" in selected_models:
            #print(t, "RidgeGraphThompson", end=" ")
                start = time.time()
                arm1=M1.choose_arm(SELECTED_USER, arms_features)
                rwd1=np.dot(arms_features[arm1], users_features[SELECTED_USER])+errors[arm1]+nu
                M1.update_B_and_y_and_mu_bar(rwd1)
                RWD1.append(np.dot(arms_features[arm1], users_features[SELECTED_USER]))
                duration = float(time.time() - start)
                computing_time[1] += duration
                record_runtime["RGraphTS"].append(duration)
            else:
                arm1 = -1

            if "LinTS" in selected_models:
                #print(t, "IndividualThompson", end=" ")
                start = time.time()
                arm2=M2.choose_arm(SELECTED_USER, arms_features)
                rwd2=np.dot(arms_features[arm2], users_features[SELECTED_USER])+errors[arm2]+nu
                M2.update_B_and_y_and_mu_bar(rwd2)
                RWD2.append(np.dot(arms_features[arm2], users_features[SELECTED_USER]))
                duration = float(time.time() - start)
                computing_time[2] += duration
                record_runtime["LinTS"].append(duration)
                #print(f"time = {round(1000*duration, 4)} ms")
            else:
                arm2 = -1

            if "SCLUB" in selected_models:
                start = time.time()
                articlePool = [Article(i, arms_features[i]) for i in range(arm_num)]
                arm3=M3.decide(articlePool, SELECTED_USER).id
                rwd3=np.dot(arms_features[arm3], users_features[SELECTED_USER])+errors[arm3]+nu
                M3.updateParameters(articlePool[arm3], rwd3, SELECTED_USER)
                RWD3.append(np.dot(arms_features[arm3], users_features[SELECTED_USER]))
                duration = float(time.time() - start)
                computing_time[3] += duration
            else:
                arm3 = -1
            arm4 = -1
            
            if "CLUB" in selected_models:
                #print(t, "CLUB", end=" ")
                start = time.time()
                arm5=M5.choose_arm(SELECTED_USER, arms_features, t)
                rwd5=np.dot(arms_features[arm5], users_features[SELECTED_USER])+errors[arm5]+nu
                M5.update_many_things(rwd5)
                RWD5.append(np.dot(arms_features[arm5], users_features[SELECTED_USER]))
                duration = float(time.time() - start)
                computing_time[5] += duration
                #print(f"time = {round(1000*duration, 4)} ms")
            else:
                arm5 = -1
                
            if "DyClu" in selected_models:
                start = time.time()
                articlePool = [Article(i, arms_features[i]) for i in range(arm_num)]
                arm6=M6.decide(articlePool, SELECTED_USER).id
                rwd6=np.dot(arms_features[arm6], users_features[SELECTED_USER])+errors[arm6]+nu
                M6.updateParameters(articlePool[arm6], rwd6, SELECTED_USER)
                RWD6.append(np.dot(arms_features[arm6], users_features[SELECTED_USER]))
                duration = float(time.time() - start)
                computing_time[6] += duration
            else:
                arm6 = -1

            if "SemiRGraphTS" in selected_models:
                #print(t, "SemiRidgeGraphThompson", end=" ")
                start = time.time()
                arm7=M7.choose_arm(SELECTED_USER, arms_features)
                rwd7=np.dot(arms_features[arm7], users_features[SELECTED_USER])+errors[arm7]+nu
                M7.update_B_and_y_and_mu_bar(rwd7)
                RWD7.append(np.dot(arms_features[arm7], users_features[SELECTED_USER]))
                duration = float(time.time() - start)
                computing_time[7] += duration
                record_runtime["SemiRGraphTS"].append(duration)
                #print(f"time = {round(1000*duration, 4)} ms")
            else:
                arm7 = -1

            if "GraphUCBlocal" in selected_models:
                #print(t, "GUCBlocal", end=" ")
                start = time.time()
                arm8=M8.choose_arm(SELECTED_USER, arms_features)
                rwd8=np.dot(arms_features[arm8], users_features[SELECTED_USER])+errors[arm8]+nu
                M8.update_user_features(rwd8)
                RWD8.append(np.dot(arms_features[arm8], users_features[SELECTED_USER]))
                duration = float(time.time() - start)
                computing_time[8] += duration
                record_runtime["GraphUCBlocal"].append(duration)
                #print(f"time = {round(1000*duration, 4)} ms")
            else:
                arm8 = -1

            if "IndividualSemiRidgeGraphThompson" in selected_models:
                #print(t, "IndividualSemiRidgeGraphThompson", end=" ")
                start = time.time()
                arm9=M9.choose_arm(SELECTED_USER, arms_features)
                rwd9=np.dot(arms_features[arm9], users_features[SELECTED_USER])+errors[arm9]+nu
                M9.update_B_and_y_and_mu_bar(rwd9)
                RWD9.append(np.dot(arms_features[arm9], users_features[SELECTED_USER]))
                duration = float(time.time() - start)
                computing_time[9] += duration
                record_runtime["IndividualSemiRidgeGraphThompson"].append(duration)
                #print(f"time = {round(1000*duration, 4)} ms")
            else:
                arm9 = -1
                
            if "LinTS_Single" in selected_models:
                #print(t, "LinTS_Single", end=" ")
                start = time.time()
                arm10=M10.choose_arm(arms_features)
                rwd10=np.dot(arms_features[arm10], users_features[SELECTED_USER])+errors[arm10]+nu
                M10.update_B_and_y_and_mu_bar(rwd10)
                RWD10.append(np.dot(arms_features[arm10], users_features[SELECTED_USER]))
                duration = float(time.time() - start)
                computing_time[10] += duration
                record_runtime["LinTS_Single"].append(duration)
                #print(f"time = {round(1000*duration, 4)} ms")
            else:
                arm10 = -1
                
            if "SemiTS_Single" in selected_models:
                #print(t, "SemiTS_Single", end=" ")
                start = time.time()
                arm11=M11.choose_arm(SELECTED_USER, arms_features)
                rwd11=np.dot(arms_features[arm11], users_features[SELECTED_USER])+errors[arm11]+nu
                M11.update_B_and_y_and_mu_bar(rwd11)
                RWD11.append(np.dot(arms_features[arm11], users_features[SELECTED_USER]))
                duration = float(time.time() - start)
                computing_time[11] += duration
                record_runtime["SemiTS_Single"].append(duration)
                #print(f"time = {round(1000*duration, 4)} ms")
            else:
                arm11 = -1

            duration_of_loop = float(time.time() - start_a_loop)
            true_opt_arm = np.argmax([np.dot(arms_features[i],users_features[SELECTED_USER]) for i in range(arm_num)])
            
            if t % time_cut == 0:
                print(t, "user:", SELECTED_USER, "/ selected arms:", arm0, arm1, arm2, arm3, arm4, arm5, arm6, arm7, arm8, arm9, arm10, arm11,"/ opt_arm:", true_opt_arm, "/ time :", round(duration_of_loop, 3), "sec")
                print("=====")
                
            optRWD.append(np.amax([np.dot(arms_features[i],users_features[SELECTED_USER]) for i in range(arm_num)]))
            
        time_for_one_iter = float(time.time() - initial)
        total_time += time_for_one_iter
        print(f"iteration = {simul+1}/{simul_n}, time = {round(time_for_one_iter, 3)} sec")
        print("==========")
        time.sleep(1)

        if "Random" in selected_models:
            cumulated_regret["Random"].append(np.cumsum(optRWD)-np.cumsum(RWD0))
        if "RGraphTS" in selected_models:
            cumulated_regret["RGraphTS"].append(np.cumsum(optRWD)-np.cumsum(RWD1))
        if "LinTS" in selected_models:
            cumulated_regret["LinTS"].append(np.cumsum(optRWD)-np.cumsum(RWD2))
        if "SCLUB" in selected_models:
            cumulated_regret["SCLUB"].append(np.cumsum(optRWD)-np.cumsum(RWD3))
        # slot4
        if "CLUB" in selected_models:
            cumulated_regret["CLUB"].append(np.cumsum(optRWD)-np.cumsum(RWD5)) 
        if "DyClu" in selected_models:
            cumulated_regret["DyClu"].append(np.cumsum(optRWD)-np.cumsum(RWD6))
        if "SemiRGraphTS" in selected_models:
            cumulated_regret["SemiRGraphTS"].append(np.cumsum(optRWD)-np.cumsum(RWD7))
        if "GraphUCBlocal" in selected_models:
            cumulated_regret["GraphUCBlocal"].append(np.cumsum(optRWD)-np.cumsum(RWD8))
        if "IndividualSemiRidgeGraphThompson" in selected_models:
            cumulated_regret["IndividualSemiRidgeGraphThompson"].append(np.cumsum(optRWD)-np.cumsum(RWD9))
        if "LinTS_Single" in selected_models:
            users_
        if "users_emiTS_Single" in selected_models:
            cumulated_regret["SemiTS_Single"].append(np.cumsum(optRWD)-np.cumsum(RWD11))
            
        for MODEL in not_selected_models:
            cumulated_regret[MODEL].append(np.zeros(time_horizon))

        #----------
    info_computing_time =""
    for i in range(0, all_models_num):
        temp = float(1000*computing_time[i]/time_horizon)
        info_computing_time = info_computing_time + all_models[i] +": "+ str(round(temp, 4)) +" ms\n"
    info_computing_time = info_computing_time + "=========================\nTOTAL TIME: "+str(round(total_time, 4))+" sec"

    print(info_computing_time) 
    
    info_computing_time_dict = {}
    for i in range(0, all_models_num):
        temp = float(1000*computing_time[i]/time_horizon)
        info_computing_time_dict[all_models[i]] = temp
    
    return cumulated_regret, info_computing_time, info_computing_time_dict, record_runtime